In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
# Page content from https://wikimon.net/Visual_List_of_Digimon

with open('page_content.txt', encoding="utf8") as f:
    content = ' '.join(l for l in f)

soup = BeautifulSoup(content)

In [ ]:
all_linked_digimon = [a['href'][1:] for table in soup.find_all('table') for a in table.find_all('a') if len(a.find_all('img')) > 0]

In [ ]:
images = {}
not_found = []

for digimon in tqdm(all_linked_digimon):
    response = requests.get("https://wikimon.net/"+digimon)
    soup = BeautifulSoup(response.text, 'html.parser')

    for link in [l for l in soup.find_all('a') if l.find('img')]:
        if link['href'].endswith('.jpg'):
            response2 = requests.get("https://wikimon.net"+link['href'])
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            
            for link2 in [l for l in soup2.find_all('a') if l.find('img')]:
                if link2['href'].endswith('.jpg'):
                    link = "https://wikimon.net"+link2['href']
                    with open('images/'+os.path.basename(link), "wb") as f:
                        f.write(requests.get(link).content)
                    images[digimon] = link2['href']
                    break
            
            if digimon in images:
                break
        if digimon in images:
            break

In [ ]:
for digimon in tqdm(images):
    link = "https://wikimon.net"+images[digimon]
    with open('images/'+os.path.basename(link), "wb") as f:
        f.write(requests.get(link).content)